In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd
#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc
from utils import *

### FNN on cleaned features

In [2]:
findex = 1
x,y = get_features(findex,train=True,cleaned=True)
print y.shape
print sum(y)
y = y.astype(int)
print sum(y)
x,y = shuffle(x,y)

(47383,)
907.0
907


In [3]:
no_ex,ip_dim = x.shape
print no_ex,ip_dim

47383 154


In [4]:
class c_mlp_1(nn.Module):
    def __init__(self):
        super(c_mlp_1,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        #self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,100)
        self.l6 = nn.Linear(100,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        #x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = F.leaky_relu(self.l6(x))
        x = (self.l7(x))
        
        return x

In [5]:
class c_mlp_2(nn.Module):
    def __init__(self):
        super(c_mlp_2,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        #self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,100)
        self.l6 = nn.Linear(100,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.tanh(self.l1(x))
        x = F.tanh(self.l2(x))
        x = F.tanh(self.l3(x))
        #x = F.leaky_relu(self.l4(x))
        x = F.tanh(self.l5(x))
        x = F.tanh(self.l6(x))
        x = (self.l7(x))
        
        return x

In [6]:
class c_mlp_3(nn.Module):
    def __init__(self):
        super(c_mlp_3,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        #self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,100)
        self.l6 = nn.Linear(100,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.selu(self.l1(x))
        x = F.selu(self.l2(x))
        x = F.selu(self.l3(x))
        #x = F.leaky_relu(self.l4(x))
        x = F.selu(self.l5(x))
        x = F.selu(self.l6(x))
        x = (self.l7(x))
        
        return x

In [7]:
mymlps = [c_mlp_1().cuda(),c_mlp_2().cuda(),c_mlp_3().cuda()]

In [9]:
for mlp_no,mymlp in enumerate(mymlps):
    max_percent = 0.90
    model_no = 1
    for i in range(1,6):
        val_iter = i
        print("val iter: ",val_iter)
        
        if model_no is 4:
                break
        else:
        #weights_array = [7]
            weights_array = np.linspace(40,60,5)

            for i,w in enumerate(weights_array): 
            
            
           
            #mymlp = c_mlp().cuda()
                optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
                criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

                for ep in range(2500):

                    ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
                    xtrain,ytrain = get_train_batch(x,y,batch_size=200,indices=ind)

                    model_op = mymlp(xtrain)

                    loss = criterion(model_op,ytrain)
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

            #     if(ep%30==29):
            #         print(loss.data[0])

                ## After training check on cross validation data
                xval,yval = get_val_data(x,y,no_examples=no_ex,val_iter=val_iter)
                min_fp = xval.size()[0]*0.17
                yval = yval.reshape(yval.shape[0],)
                train_op = mymlp(xval)
                train_op = train_op.cpu().data.numpy()
                pred_labels = np.argmax(train_op,axis=1)

                cf = metrics.confusion_matrix(yval,pred_labels).ravel()
                [tn,fp,fn,tp] = cf
                print('tn, fp, fn, tp: ',cf)
                percent = float(float(tp)/float(tp+fn))
                if(percent>max_percent*0.98):
                    if(fp<min_fp):
                        if(1==1):
                                max_percent = percent
                                print("min fp, max_percent",fp,percent)
                                model_path = os.getcwd() + '/fnn_' + str(findex)+'_model'+str(model_no)+'_mlp'+str(mlp_no)
                                torch.save(mymlp.state_dict(),model_path)
                                print("saving model on val: ",val_iter," and weight: ",w)
                                model_no = model_no + 1





('val iter: ', 1)
('tn, fp, fn, tp: ', array([9289,    7,    0,  180]))
('min fp, max_percent', 7, 1.0)
('saving model on val: ', 1, ' and weight: ', 40.0)
('tn, fp, fn, tp: ', array([9287,    9,    2,  178]))
('min fp, max_percent', 9, 0.9888888888888889)
('saving model on val: ', 1, ' and weight: ', 45.0)
('tn, fp, fn, tp: ', array([9289,    7,   10,  170]))
('tn, fp, fn, tp: ', array([9287,    9,   13,  167]))
('tn, fp, fn, tp: ', array([9288,    8,   12,  168]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([9274,    9,    0,  194]))
('min fp, max_percent', 9, 1.0)
('saving model on val: ', 2, ' and weight: ', 40.0)
('tn, fp, fn, tp: ', array([9277,    6,    4,  190]))
('tn, fp, fn, tp: ', array([9270,   13,    8,  186]))
('tn, fp, fn, tp: ', array([9269,   14,   12,  182]))
('tn, fp, fn, tp: ', array([9262,   21,   16,  178]))
('val iter: ', 3)
('val iter: ', 1)
('tn, fp, fn, tp: ', array([7915, 1381,   65,  115]))
('tn, fp, fn, tp: ', array([7313, 1983,   39,  141]))
('tn, fp, fn,

In [13]:
xtest_t,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest_t).cuda())
final_pred = np.zeros(ytest.shape[0],)
print final_pred.shape

test_models = [c_mlp_1().cuda(),c_mlp_2().cuda(),c_mlp_3().cuda()]
#     for i in range(2):
#         test_models.append(model)
for mlp_no,test_model in enumerate(test_models):
    for model_no in range(1,4):
        model_path = os.getcwd() + '/fnn_' + str(findex)+'_model'+str(model_no)+'_mlp'+str(mlp_no)

        test_model.load_state_dict(torch.load(model_path))
        test_op = test_model(xtest)
        test_op = test_op.cpu().data.numpy()
        pred_labels = np.argmax(test_op,axis=1).reshape(final_pred.shape[0],)
        print pred_labels.shape
        final_pred = pred_labels + final_pred
        print final_pred.shape
        cf = metrics.confusion_matrix(ytest,pred_labels).ravel()
        print('tn, fp, fn, tp: ',cf)

(11957,)
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11788,   127,    37,     5]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11782,   133,    37,     5]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11790,   125,    36,     6]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11536,   379,    31,    11]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11669,   246,    33,     9]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11696,   219,    34,     8]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11092,   823,    22,    20]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11510,   405,    33,     9]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([11654,   261,    33,     9]))


In [11]:
final_pred[final_pred>1] = 1
cf = metrics.confusion_matrix(ytest,final_pred).ravel()
print('tn, fp, fn, tp: ',cf)

('tn, fp, fn, tp: ', array([10940,   975,    19,    23]))


In [ ]:
pred_labels.shape

In [ ]:
final_pred.shape